### Import Library yang akan digunakan

In [37]:
import pandas as pd
import re
import string
import nltk
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [38]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Memuat Data Tweet

In [39]:
link = "https://raw.githubusercontent.com/HaikalAkbar13/dicoding_sentimen_analysis/refs/heads/main/tweets_data.csv"
df = pd.read_csv(link)
df.columns

Index(['Tweet ID', 'Tweet Time Stamp', 'Tweet Text', 'Tweet Hashtag',
       'Tweet Translated'],
      dtype='object')

### Assesing dan Cleaning Data

In [40]:
# Drop kolom yang tidak di butuhkan (Timestamp, Tweet Text, Hashtag)
df = df.drop(columns=['Tweet Time Stamp', 'Tweet Text', 'Tweet Hashtag'])
df.head()

,Tweet ID,Tweet Translated
0,1609172529712611329,The Embark Internship has allowed our interns ...
1,1609243867504590850,asked ChatGPT: What jobs will be needed for hu...
2,1609261573184581635,"Thanks to ChatGPT, I just learned a new skill!..."
3,1609043258625216512,"Maybe AI won't be taking our jobs... at least,..."
4,1608679261136912384,"Well, @openai #ChatGPT completely shit the bed..."


In [41]:
# Duplicate Value
print(f"Jumlah Duplicated Value sebelum di drop \n{df.duplicated().value_counts()}")
df = df.drop_duplicates()
print(f"Jumlah Duplicated Value setelah di drop \n{df.duplicated().value_counts()}")

Jumlah Duplicated Value sebelum di drop 
False    14073
True        33
Name: count, dtype: int64
Jumlah Duplicated Value setelah di drop 
False    14073
Name: count, dtype: int64


In [42]:
# Null Values
for column in df.columns:
    print(f"Jumlah Null Values pada kolom {column} : {df[column].isna().value_counts()}")

Jumlah Null Values pada kolom Tweet ID : Tweet ID
False    14073
Name: count, dtype: int64
Jumlah Null Values pada kolom Tweet Translated : Tweet Translated
False    14073
Name: count, dtype: int64


### Pre Processing Data

In [48]:
# Membuat Fungsi untuk Pre processing Data
def preprocessing(text):
    # Inisialisasi
    punctuation_set = set(string.punctuation)
    stop_words = set(stopwords.words('english'))
    lemma = WordNetLemmatizer()
    text = text.lower()
    # Strip whitespaces
    text = text.strip()
    # Hapus URL, mentions, hashtag, newline, angka, special characters
    text = re.sub(r"http\S+|www\.\S+|@\w+|#\w+|[^\d\w\S]|\d+|\W|\0|_+|[^\x00-\x7F]+", " ", text)
    # Hapus punctuation
    text = ''.join(char for char in text if char not in punctuation_set)
    # Tokenisasi dan hapus stopwords
    tokens = word_tokenize(text)
    filtered = [word for word in tokens if word not in stop_words]
    # Lemmatization
    text = [lemma.lemmatize(word) for word in filtered]
    text = " ".join(word for word in text)
    return text

In [49]:
df["Text Cleaned"] = df["Tweet Translated"].apply(preprocessing)
df.head(30)

,Tweet ID,Tweet Translated,Text Cleaned
0,1609172529712611329,The Embark Internship has allowed our interns ...,embark internship allowed intern grow professi...
1,1609243867504590850,asked ChatGPT: What jobs will be needed for hu...,asked chatgpt job needed human perform artific...
2,1609261573184581635,"Thanks to ChatGPT, I just learned a new skill!...",thanks chatgpt learned new skill
3,1609043258625216512,"Maybe AI won't be taking our jobs... at least,...",maybe ai taking job least yet
4,1608679261136912384,"Well, @openai #ChatGPT completely shit the bed...",well completely shit bed one pharmacologist bi...
5,1608963021824561154,“AI Rewrite: Steve Jobs Stanford Commencement”...,ai rewrite steve job stanford commencement pet...
6,1609304139116302339,🧵 How to replace bullshit jobs with AI 🧵 \n\n2...,replace bullshit job ai year never seen risk o...
7,1609301436428423174,Four Ways #Jobs Will Respond to #Automation \n...,four way respond via
8,1608996363123007489,BGSU program to meet critical workforce needs ...,bgsu program meet critical workforce need adva...
9,1608952318787026947,New books from MIT experts deliver insights on...,new book mit expert deliver insight future rea...


### Labeling Data

In [50]:
# Labeling data menggunakan Model dari Hugging Face
label = ["Positive", "Neutral", "Negative"]
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
cls = pipeline("sentiment-analysis",
                      model=model_name, tokenizer=model_name)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [55]:
def labelling(text):
  result = []
  for tweet in text:
    model_predict = cls(tweet)
    result.append(model_predict[0]["label"])
  return result

In [ ]:
df["Sentiment"] = df["Text Cleaned"].apply(labelling)
df.head()